# Miscellaneous notes

## Data source and idea

From the github of HANNA: https://github.com/dig-team/hanna-benchmark-asg, 
we can retrieve the file hanna_stories_annotation.csv, which contains, for 96 prompts, a story generated by a human and a story generated by 10 ASG systems, so 1056 stories in total. 

The idea is to try to reproduce, to some extent the results depicted in "Of human criteria and automatic metrics: a benchmark of the evaluation of story generation"

## Various metrics

* From https://github.com/PierreColombo/nlg_eval_via_simi_measures: DepthScore, BaryScore, InfoLM
* From https://github.com/neural-dialogue-metrics/BLEU: BLEU
* From NLTK : BLEU
* From https://github.com/neural-dialogue-metrics/rouge: ROUGE
* From https://github.com/pltrdy/rouge : ROUGE (alternative implementation)
* From https://github.com/bheinzerling/pyrouge  pyrouge : ROUGE (Rouge155)
* : METEOR





# Dependencies


## Installations

In [ ]:
#!pip install -r ../requirements.txt

In [ ]:
!pip install transformers
!pip install torch

In [ ]:
!pip install nltk
!pip install pyrouge

In [ ]:
!pip install git+https://github.com/PierreColombo/nlg_eval_via_simi_measures.git

In [ ]:
#!pip install git+https://github.com/neural-dialogue-metrics/BLEU.git
!pip install git+https://github.com/neural-dialogue-metrics/rouge.git
!pip install git+https://github.com/pltrdy/rouge

## Dependencies

In [ ]:
import numpy as np 
import sklearn
import torch 
import transformers
from tqdm import tqdm

In [ ]:
from nlg_eval_via_simi_measures.bary_score import BaryScoreMetric
from nlg_eval_via_simi_measures.depth_score import DepthScoreMetric
from nlg_eval_via_simi_measures.infolm import InfoLM

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

# Testing metrics

In [ ]:
ref = ['I like my cakes very much','I hate these cakes!']
hypothesis = ['I like my cakes very much','I like my cakes very much']

# Baryscore

def compute_bary(ref, hypothesis):
  metric_call = BaryScoreMetric()
  metric_call.prepare_idfs(ref, hypothesis)
  final_preds = metric_call.evaluate_batch(ref, hypothesis)
  print(final_preds)
  print("="*25)

compute_bary(ref, hypothesis)

ref = ['I like my cakes very much']
hypothesis = ['I like my cakes very much']

compute_bary(ref, hypothesis)

ref = ['I hate these cakes!']
hypothesis = ['I like my cakes very much']

compute_bary(ref, hypothesis)


*   W c'est pour distance de Wasserstein
*   SD, c'est pour Sinkhorn Divergence, donc autre chose et a priori pas ce qui nous intéresse (pour différentes valeurs d'un paramètre)

Donc, a priori, on prend juste "baryscore_W"

* Il faut calculer les idf au niveau du corpus, mais d'après ce que je lis dans l'article, ca se fait séparemment pour chaque paire référence/candidat (en tout cas le contraire n'est pas clairement indiqué)

Donc, à mon avis, même s'il y a une fonction "evaluate_batch", on doit faire les calculs individuellement pour chaque pair, et pas tout concaténer. Mais le calcul de l'idf doit se faire au niveau des **textes** complets que l'on compare. 

Plus la métrique est proche de 0, mieux c'est. 



In [ ]:
# DepthScore

ref = ['I like my cakes very much','I hate these cakes!']
hypothesis = ['I like my cakes very much','I like my cakes very much']

metric_call = DepthScoreMetric()
metric_call.prepare_idfs(ref, hypothesis)
final_preds = metric_call.evaluate_batch(hypothesis,ref )
print("DepthScore")
print(final_preds)


Calcul d'un embedding pour le candidat et la référence avec a single layer of Bert (comme Bertscore apparemment) puis calcule la divergence introduite dans "a pseudo metric", donc plus c'est proche de 0 et plus les deux phrases sont proches

In [ ]:
# Pour l'instant, ca fonctionne uniquement si on se met sur CPU (il manque surement un .todevice() quelque part)

# InfoLM 

metric = InfoLM()

ref = ['I like my cakes very much','I hate these cakes!']
hypothesis = ['I like my cakes very much','I like my cakes very much']

metric.prepare_idfs(ref, hypothesis)

final_preds = metric.evaluate_batch(hypothesis, ref)#), 
                                    #idf_ref = metric.idf_dict_hyp,
                                    #idf_hyps = metric.idf_dict_ref)
                                    #idf_ref= idf_ref,
                                    #idf_hyps= idf_hypot)

#self.idf_dict_hyp, self.idf_dict_ref 
print(final_preds)

Encore une fois, il me semble qu'on calcule les idf à l'échelle d'une pair candidat (C) / référence (R). 
D'abord, séparemment pour C et R, le PMLM calcule une probabilité discrète par token en le masquant, puis on l'agrège en mettant des poids via les idf, puis on utilise une mesure de divergence pour estimer la distance entre les deux distributions de probabilité obtenues. 

Par défaut, on utilise Fisher Rao qui, d'après l'article, est pas mal et a l'avantage de ne pas inclure de paramètre à optimiser. 

On a 3 résultats, parce que l'algo permet d'utiliser des divergences, qui ne sont pas symétriques. Du coup le premier c'est Div(A,B), le second Div(B,A) et le 3ème la moyenne des deux. Mais vu qu'on utilise la DISTANCE de Fisher Rao (qui évidemment est une distance), elle est symétrique et osef